In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_iris


# Import PySwarms
import pyswarms as ps

# Some more magic so that the notebook will reload external python modules;
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

In [2]:
f= open("two_spirals.dat","r")
data = np.loadtxt(f)

In [3]:
# Store the features as X and the labels as y
X = data[:,0:2]
y = data[:,2]
#print((X))

In [4]:
n_inputs = 2
n_hidden = 8
n_classes = 2

num_samples = (n_inputs*n_hidden)+(n_hidden*n_classes)+n_hidden+n_classes
print(num_samples)

42


In [5]:
def logits_function(p):
    """ Calculate roll-back the weights and biases

    Inputs
    ------
    p: np.ndarray
        The dimensions should include an unrolled version of the
        weights and biases.

    Returns
    -------
    numpy.ndarray of logits for layer 2

    """
    # Roll-back the weights and biases
    W1 = p[0:n_inputs*n_hidden].reshape((n_inputs,n_hidden))
    b1 = p[n_inputs*n_hidden:n_inputs*n_hidden+n_hidden].reshape((n_hidden,))
    W2 = p[n_inputs*n_hidden+n_hidden:n_inputs*n_hidden+n_hidden+n_hidden*n_classes].reshape((n_hidden,n_classes))
    b2 = p[n_inputs*n_hidden+n_hidden+n_hidden*n_classes:n_inputs*n_hidden+n_hidden+n_hidden*n_classes+n_classes].reshape((n_classes,))

    # Perform forward propagation
    z1 = X.dot(W1) + b1  # Pre-activation in Layer 1
    a1 = np.tanh(z1)     # Activation in Layer 1
    logits = a1.dot(W2) + b2 # Pre-activation in Layer 2
    return logits          # Logits for Layer 2

In [8]:
def forward_prop(params):
    """Forward propagation as objective function

    This computes for the forward propagation of the neural network, as
    well as the loss.

    Inputs
    ------
    params: np.ndarray
        The dimensions should include an unrolled version of the
        weights and biases.

    Returns
    -------
    float
        The computed negative log-likelihood loss given the parameters
    """

    logits = logits_function(params)

    # Compute for the softmax of the logits
    exp_scores = np.exp(logits)
    probs = exp_scores / np.sum(exp_scores, axis=1, keepdims=True)
 #   print("hello\n")
    #print(logits)
    # Compute for the negative log likelihood
#     correct_logprobs =np.zeros(num_samples)
#     for i in range(num_samples):
#         correct_logprobs[i] = -np.log(probs[i])
#  #   corect_logprobs = -np.log(probs[range(num_samples), y])
#   #  corect_logprobs= [1,2,3,4,5,1]
    corect_logprobs = logits
    loss = np.sum(corect_logprobs) / num_samples

    return loss

In [9]:

def f(x):
    """Higher-level method to do forward_prop in the
    whole swarm.

    Inputs
    ------
    x: numpy.ndarray of shape (n_particles, dimensions)
        The swarm that will perform the search

    Returns
    -------
    numpy.ndarray of shape (n_particles, )
        The computed loss for each particle
    """
    n_particles = x.shape[0]
    j = [forward_prop(x[i]) for i in range(n_particles)]
    return np.array(j)

In [10]:
%%time
# Initialize swarm
options = {'c1': 0.5, 'c2': 0.3, 'w':0.9}

# Call instance of PSO
dimensions = (n_inputs * n_hidden) + (n_hidden * n_classes) + n_hidden + n_classes
optimizer = ps.single.GlobalBestPSO(n_particles=100, dimensions=dimensions, options=options)

# Perform optimization
cost, pos = optimizer.optimize(f, iters=1000)

2020-11-10 00:35:58,492 - pyswarms.single.global_best - INFO - Optimize for 1000 iters with {'c1': 0.5, 'c2': 0.3, 'w': 0.9}
pyswarms.single.global_best: 100%|██████████|1000/1000, best_cost=-2.89e+16
2020-11-10 00:36:10,095 - pyswarms.single.global_best - INFO - Optimization finished | best cost: -2.891998407250301e+16, best pos: [-2.10832075e+01 -4.28634734e+00 -1.99788625e+01 -1.38412645e+01
 -1.20861846e+00 -2.04495011e+00  7.81817130e-01  4.62340963e-01
 -2.41813492e+00 -8.53415011e+00 -1.78642901e+01 -5.43431921e+00
  4.75984361e+00  3.26955666e+00 -1.19034677e+01 -2.95509776e-01
 -6.53190335e+00 -7.64688771e+00  3.32013886e+00 -1.70960517e+00
  2.05451421e+01  1.08908895e+01 -3.24631273e+00  4.96451746e+00
 -5.14049219e+00 -1.92410892e+01 -3.37018261e+00 -7.07056512e+00
 -1.32821467e+00 -1.50716592e+01 -1.32080123e+01 -3.14729705e+00
 -6.14452017e+01 -3.88613286e+01  2.42119914e+00 -5.38602807e+00
 -6.94725323e+00  4.99534800e+00 -9.20044201e+00 -4.60388502e+15
 -1.35408290e+02 

In [11]:
def predict(pos):
    """
    Use the trained weights to perform class predictions.

    Inputs
    ------
    pos: numpy.ndarray
        Position matrix found by the swarm. Will be rolled
        into weights and biases.
    """
    logits = logits_function(pos)
    y_pred = np.argmax(logits, axis=1)
    return y_pred

In [12]:
(predict(pos) == y).mean()

0.5